In [1]:
%pip install torch torchvision torchaudio


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install transformers requests 
%pip install beautifulsoup4 as bs4
%pip install pandas as pd 
%pip install numpy as np

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
#tokenzier allows us to pass the string and conver it into the sequences of arrays

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Instantiate Model


In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model=AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Encode and Calculate Sentiment


In [5]:
tokens=tokenizer.encode('I hated this, absolutley the worst',return_tensors='pt')
#pt refer to the format it is expected to return the token and in this case it is pytorch tensor, if tensorflow then tf

In [6]:
tokens[0]

tensor([  101,   151, 39487, 10163, 10372,   117, 79244, 11988, 10103, 43060,
          102])

In [7]:
tokenizer.decode(tokens[0])

'[CLS] i hated this, absolutley the worst [SEP]'

In [8]:
result=model(tokens)

Here below code we can see that the most important part is logits=tensor. It returns 5 different values signlaing 5 different ratings of the sentiments


In [9]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.6157,  1.6438, -0.6631, -2.7900, -2.0412]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [10]:
result.logits

tensor([[ 4.6157,  1.6438, -0.6631, -2.7900, -2.0412]],
       grad_fn=<AddmmBackward0>)

In [11]:
torch.argmax(result.logits)

tensor(0)

Here we are getting 0 Since our rating is from 1 to 5 we can convert to integers


In [12]:
sentiments=int(torch.argmax(result.logits))+1
sentiments

1

In [13]:
tokens=tokenizer.encode("This is absolutely amazing, wow",return_tensors='pt')

In [14]:
results=model(tokens)

In [15]:
results

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5625, -2.3823, -1.1381,  0.4518,  3.9948]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [16]:
int(torch.argmax(results.logits))+1

5

In [17]:
tokens=tokenizer.encode("It' okay",return_tensors='pt')

In [18]:
result=model(tokens)

In [19]:
int(torch.argmax(result.logits))+1

3

Using transformer to translate nepali sentences written in english to English sentences


In [20]:
from transformers import BertForMaskedLM, pipeline

In [26]:
tokenizer1=AutoTokenizer.from_pretrained('Rajan/NepaliBERT',strip_accents=False,clean_text=False)
model_nepali = BertForMaskedLM.from_pretrained('Rajan/NepaliBERT')

strip_accent and clean_text are made false because by default the tokenzier removes them and since we are translating nepali sentences puncutation and accent is important to recognize the true meaning of the sentences


In [27]:
# Create a fill-mask pipeline
fill_mask = pipeline("fill-mask", model=model_nepali, tokenizer=tokenizer)

# Use the pipeline to generate translations
result = fill_mask("Mero nam [MASK] ho")
print(result[0]['sequence'])

mero nam ho


In [25]:
results

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5625, -2.3823, -1.1381,  0.4518,  3.9948]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [52]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [53]:
reviews

[]

Loading Reviews into Dataframe and Score


In [54]:
import pandas as pd 
import numpy as np 


In [59]:
df=pd.DataFrame(np.array(reviews),columns=['review'])

In [60]:
df.head()

,review


In [61]:
df.tail()

,review


In [62]:
df['review'].iloc[0]

IndexError: single positional indexer is out-of-bounds

In [63]:
def sentimnet_score(review):
    tokens=tokenizer.encode(review, return_tensors='pt')
    result=model(tokens)
    return int(torch.argmax(result.logits))+1

In [64]:
sentimnet_score(df['review'].iloc[1])

IndexError: single positional indexer is out-of-bounds

In [65]:
df['sentimnet']=df['review'].apply(lambda x:sentimnet_score(x[:512]))

In [66]:
df

,review,sentimnet


In [67]:
df['review'].iloc[3]

IndexError: single positional indexer is out-of-bounds